In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.ops import box_convert
from source_code.card_detector import CardDetector, fit
from source_code.dataloader_utils import MTGCardsDataset, get_transform_pipe
from source_code.config import *
import wandb

In [2]:
SOURCE_PATH = CONFIG["training"]["SOURCE_PATH"]
DEST_PATH = CONFIG["training"]["DEST_PATH"]

In [3]:
anchor_boxes = torch.Tensor(CONFIG["training"]["ANCHOR_BOXES"]) # Anchor boxes acquired from k-means clustering of the dataset

model = CardDetector(
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  anchor_boxes=anchor_boxes,
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  head='v3'
)
if SOURCE_PATH is not None:
    model.load_state_dict(torch.load(SOURCE_PATH))

feature_map_dims = (model.features_w, model.features_h)

transform_pipe = get_transform_pipe(img_w=CONFIG["dataset"]["img_w"], img_h=CONFIG["dataset"]["img_h"])

train_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_train"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=CONFIG["dataset"]["num_max_boxes"],
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
train_dataloader = DataLoader(train_dataset, batch_size=CONFIG["dataloader"]["batch_size"])

val_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_val"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=CONFIG["dataset"]["num_max_boxes"],
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
val_dataloader = DataLoader(val_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

test_dataset = MTGCardsDataset(
  annotations_file=CONFIG["dataset"]["annotations_file_test"],
  img_dir=CONFIG["dataset"]["img_dir"], #TODO: change directory when we have the actual data
  anchor_boxes=model.anchor_boxes,
  feature_map_dims=feature_map_dims,
  img_dims= (CONFIG["dataset"]["img_w"], CONFIG["dataset"]["img_h"]),
  num_anchors_per_cell=CONFIG["dataset"]["num_anchors_per_cell"],
  num_max_boxes=CONFIG["dataset"]["num_max_boxes"],
  transform=transform_pipe,
  limit=CONFIG["dataset"]["limit"]
)
test_dataloader = DataLoader(test_dataset, batch_size=CONFIG["dataloader"]["batch_size"]) 

In [4]:
if WANDB_LOGGING:
    wandb.init(project=CONFIG['project_name'])
opt = torch.optim.Adam(params=model.parameters(), lr=CONFIG["optimizer"]["lr"])
fit(
    model=model,
    num_epochs=CONFIG["optimizer"]["num_epochs"],
    optimizer=opt, 
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=model.device,
    localization_weight=CONFIG["training"]["LOCALIZATION_WEIGHT"],
    save_path=DEST_PATH
)
if WANDB_LOGGING:
    wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vikioza (fiit-nn-2023-lp-vs). Use `wandb login --relogin` to force relogin


  0%|          | 0/20 [00:00<?, ?it/s]


Epoch: 0


c:\Users\vikio\anaconda3\envs\nsiete_pytorch_project\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Looked at 16 Batches	---	1000/1000 Samples
Train Total Loss:    1.5068932995200157
Train Objectness Loss:   0.5795831400901079
Train Localization Loss: 0.9273101650178432
Val Total Loss:          1.0573624297976494
Val Objectness Loss:     0.5230606980621815
Val Localization Loss:   0.5343017242848873

Epoch: 1
Looked at 16 Batches	---	1000/1000 Samples
Train Total Loss:    0.7888402044773102
Train Objectness Loss:   0.42701535671949387
Train Localization Loss: 0.3618248403072357
Val Total Loss:          0.7199097685515881
Val Objectness Loss:     0.3478526957333088
Val Localization Loss:   0.3720570709556341

Epoch: 2
Looked at 16 Batches	---	1000/1000 Samples
Train Total Loss:    0.6189899332821369
Train Objectness Loss:   0.33190469816327095
Train Localization Loss: 0.2870852369815111
Val Total Loss:          0.586013201624155
Val Objectness Loss:     0.2795162443071604
Val Localization Loss:   0.3064969554543495

Epoch: 3
Looked at 16 Batches	---	1000/1000 Samples
Train Total Loss:

Train Localization Loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Objectness Loss,█▆▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Train Total Loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Val Localization Loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val Objectness Loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
Val Total Loss,█▅▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
Train Localization Loss,0.20325
Train Objectness Loss,0.10398
Train Total Loss,0.30723
Val Localization Loss,0.25458
Val Objectness Loss,0.10891
